<a href="https://colab.research.google.com/github/LeeMinJun0102/Car_Price_Predict/blob/main/ML_prepro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.font_manager as fm
import ast

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, MultiLabelBinarizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import plot_tree, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_excel('usedcar2019.xlsx')
df.info()

In [ ]:
df.head()

In [ ]:
df = df.rename(columns={
    '판매일자': 'sales_date',
    '지점': 'branch',
    '원담당자': 'original_manager',
    '판매사원번호': 'salesperson_id',
    '판매사원': 'salesperson',
    '제조사': 'manufacturer',
    '모델': 'model',
    '대분류': 'category_major',
    '소분류': 'category_minor',
    '차량번호': 'car_number',
    '연식': 'year',
    '연식2': 'year2',
    '상세모델': 'model_detail',
    '신차가격': 'new_car_price',
    '잔차율': 'residual_value_rate',
    '잔차체크': 'residual_check',
    '차량가격': 'car_price',
    '매입가': 'purchase_price',
    'PM매입가': 'PM_purchase_price',
    '광고가': 'advertised_price',
    '판매가': 'sales_price',
    'PM판매가(최초)': 'PM_sales_price_initial',
    'PM판매가(현재)': 'PM_sales_price',
    '상품화금액(부가세제외)': 'refurbishment_cost_ex_vat',
    '누락상품화금액(부가세제외)': 'missing_refurbishment_cost_ex_vat',
    '파워딜러수수료': 'power_dealer_fee',
    '보정이익금': 'adjusted_profit',
    '프로모션비용': 'promotion_cost',
    '이익금': 'profit',
    '카드수수료': 'card_fee',
    '딜러연계수수료': 'dealer_link_fee',
    'K Car 홈서비스 탁송비': 'kcar_home_delivery_fee',
    '재고비용1(주차비)': 'inventory_cost_parking',
    '재고비용2(금융비)': 'inventory_cost_finance',
    'PM판매가 기준 예상이익': 'expected_profit_based_on_PM_sales',
    '보증구분': 'warranty_type',
    '보증금액': 'warranty_amount',
    '보험수수료': 'insurance_fee',
    '할부수수료': 'installment_fee',
    '이익률': 'profit_margin',
    '권장판매기간': 'recommended_sales_period',
    '판매기간': 'sales_period',
    '재고유형': 'inventory_type',
    '판매채널': 'sales_channel',
    '매입일자': 'purchase_date',
    '매입사원번호': 'purchaser_id',
    '매입사원': 'purchaser',
    '진단사원': 'inspector',
    '기여사원': 'contributor',
    '매입1채널': 'purchase_channel_1',
    '매입2채널': 'purchase_channel_2',
    '업체명': 'company_name',
    '상품화금액(부가세포함)': 'refurbishment_cost_incl_vat',
    '누락상품화금액(부가세포함)': 'missing_refurbishment_cost_incl_vat',
    'PM산출상품화(부가세제외)': 'PM_refurbishment_ex_vat',
    '매입원가': 'purchase_cost',
    '매입전차량번호': 'previous_car_number',
    '연식.1': 'year_alt',
    '주행거리': 'mileage',
    '색상': 'color',
    '연료': 'fuel',
    '차종': 'car_type',
    '매입사고': 'purchase_accident',
    '진단사고': 'inspection_accident',
    '변속기': 'transmission',
    '세금계산서여부': 'tax_invoice',
    '차량ID': 'car_id',
    '계약상태': 'contract_status',
    '판매시광고': 'sales_ad',
    '닷컴광고여부': 'dotcom_ad',
    'PM준수여부(매입)': 'pm_compliance_purchase',
    'PM준수여부(판매)': 'pm_compliance_sales',
    '조회수(엔카닷컴)': 'views_encar',
    '조회수(K Car)': 'views_kcar',
    '매입취득세': 'acquisition_tax',
    '유레카프로': 'eureka_pro',
    '신규지점 재고이동': 'inventory_transfer_new_branch',
    '대형지점': 'large_branch',
    '판매수수료': 'sales_fee',
    '낙찰수수료': 'auction_fee',
    '출품수수료': 'listing_fee',
    '매입미준수사유': 'non_compliance_reason',
    '장기재고율': 'long_term_inventory_rate',
    '무조건승인여부': 'unconditional_approval',
    '자동인여부': 'auto_approval',
    '매입불공제여부': 'purchase_non_deductible'
})

In [ ]:
df.head()

In [ ]:
# sns.pairplot(df)

In [ ]:
# 연도와 월을 숫자로 추출
df['year_num'] = df['year'].str[:4].astype(int)       # 앞 4자리: 연도
df['month_num'] = df['year'].str[-2:].astype(int)     # 뒤 2자리: 월

# 사용연수 계산 (2019년 기준 + 개월을 년 단위로 변환)
df['usedyear'] = (2019 - df['year_num']) + (2/12 - df['month_num']/12)

# 판매채널 grouping 파생변수 생성
new_sold = []
for i in df['sales_channel']:
    if i in ['K Car 옥션', '서울경매장', '글로비스(분당)']:
        new_sold.append('경매')
    elif i in ['K Car 모바일', '내차사기 홈서비스', 'K Car 고객', '엔카닷컴(모바일)', '엔카닷컴고객']:
        new_sold.append('온라인 고객')
    elif i == '내방고객':
        new_sold.append('내방고객')
    else:
        new_sold.append('기타')

df['sales_channel_grouping'] = new_sold

# 매입채널 grouping 파생변수 생성
new_buy = []
for j in df['purchase_channel_1']:
    if j in ['법인/리스', '렌트영업', '제휴영업', '경매장영업', '일반법인', '렌터카', '법인수입차영업']:
        new_buy.append('법인')
    elif j == '영업소':
        new_buy.append('영업소')
    elif j in ['개인', '대차']:
        new_buy.append('개인')
    elif j == '내차팔기 홈서비스':
        new_buy.append('내차팔기 홈서비스')
    else:
        new_buy.append('기타')

df['purchase_channel_1_grouping'] = new_buy

# 연료 grouping
fuel_mapping = {
    '가솔린+전기': '전기',
    '가솔린+LPG': 'LPG',
    'LPG(일반인 구입)': 'LPG',
    'LPG+전기': '전기'
}

df['fuel_grouping'] = df['fuel'].replace(fuel_mapping)

# 변속기 grouping
trans_mapping = {
    '세미오토': '오토',
    'CVT': '오토'
}

df['transmission_grouping'] = df['transmission'].replace(trans_mapping)

# 제조사 grouping
M_grouping = []
for i in  df['manufacturer']:
    if i in ['벤츠', 'BMW', '아우디', '폭스바겐', '미니']:
        M_grouping.append('독일')
    elif i in ['도요타', '혼다', '렉서스', '닛산', '인피니티', '미쯔비시', '스즈키']:
        M_grouping.append('일본')
    elif i in ['현대']:
        M_grouping.append('현대')
    elif i in ['기아']:
        M_grouping.append('기아')
    elif i in ['쉐보레(GM대우)']:
        M_grouping.append('쉐보레(GM대우)')
    elif i in ['쌍용']:
        M_grouping.append('쌍용')
    elif i in ['르노삼성']:
        M_grouping.append('르노삼성')
    else:
        M_grouping.append('기타')

df['manufacturer_grouping'] = M_grouping

# 색상 grouping
C_grouping = []
for i in  df['color']:
    if i in ['흰색']:
        C_grouping.append('흰색')
    elif i in ['검정색']:
        C_grouping.append('검정색')
    elif i in ['은색']:
        C_grouping.append('은색')
    elif i in ['쥐색']:
        C_grouping.append('쥐색')
    elif i in ['진주색']:
        C_grouping.append('진주색')
    else:
        C_grouping.append('기타')

df['color_grouping'] = C_grouping

# 차종 grouping
mapping = {
    '소형차': '준중형차',
    'RV': 'SUV'
}

df['car_type_grouping'] = df['car_type'].replace(mapping)

In [ ]:
df.head()

# grouping 전 후 행 개수 비교 필요

In [ ]:
df['sales_channel'].value_counts()

In [ ]:
df['sales_channel_grouping'].value_counts()

In [ ]:
df['purchase_channel_1'].value_counts()

In [ ]:
df['purchase_channel_1_grouping'].value_counts()

In [ ]:
df['fuel'].value_counts()

In [ ]:
df['fuel_grouping'].value_counts()

In [ ]:
df['transmission'].value_counts()

In [ ]:
df['transmission_grouping'].value_counts()

In [ ]:
df['manufacturer'].value_counts()

In [ ]:
df['manufacturer_grouping'].value_counts()

In [ ]:
df['color'].value_counts()

In [ ]:
df['color_grouping'].value_counts()

In [ ]:
df['car_type'].value_counts()

In [ ]:
df['car_type_grouping'].value_counts()

In [ ]:
df[df['sales_price'] == 0]

In [ ]:
df['purchase_accident'].value_counts()

In [ ]:
df[df['purchase_accident'] == '사고']

In [ ]:
df['car_type'].value_counts()

In [ ]:
df = df[df['sales_price'] != 0]

In [ ]:
df = df[df['purchase_accident'] != '사고']

In [ ]:
df = df[~df['car_type'].isin(['기타', '경승합차', '버스'])]

In [ ]:
import pandas as pd
import pandas as pd

# 엑셀로 저장
output_path = "/content/Final.xlsx"  # 경로 원하는 대로 수정 가능
df.to_excel(output_path, index=True)

print(f"✅ 저장 완료: {output_path}")